# BiOrthogonal Basis Decomposition

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse

from tomotok.core.geometry import sparse_line_3d, generate_los, RegularGrid
from tomotok.core.phantoms import gauss_iso
from tomotok.core.inversions import Bob, SparseBob, CholmodBob

### Phantom and Grid

In [ ]:
nr, nz = 50, 60
grid = RegularGrid(nr, nz, (.2, .7), (-.3, .3))
phantom = gauss_iso(grid.nr, grid.nz, cen=.3, w=.05) * 2000
phantom = np.roll(phantom, (3, -5), (0, 1))

In [ ]:
plt.figure()
plt.imshow(phantom, origin='lower', extent=grid.extent)
plt.colorbar(label='Emissivity [-]')
plt.xlabel('r [m]')
plt.ylabel('Z [m]')
plt.show()

### Artificial Camera Setup

In [ ]:
pinhole = (.8, -.1, .2)
resolution = (60, 60)
# resolution = (800, 800)  # higher resolutions requires significant amount of memory
fov = (40, 40)
axis = (-1, -.3, -.3)

In [ ]:
start, end = generate_los(pinhole=pinhole, num=resolution, fov=fov, axis=axis, elong=3)
# reorganize for geometry matrix computation
npix = resolution[0] * resolution[1]
xch = np.zeros((npix, 2))
ych = np.zeros((npix, 2))
zch = np.zeros((npix, 2))
xch[:, 0] = start[:, 0]
xch[:, 1] = end[:, 0]
ych[:, 0] = start[:, 1]
ych[:, 1] = end[:, 1]
zch[:, 0] = start[:, 2]
zch[:, 1] = end[:, 2]

### Geometry Matrix

In [ ]:
gmat = sparse_line_3d(xch, zch, grid, ych, rmin=.2, step=0.005)

### Synthetic Image

In [ ]:
image = gmat.dot(phantom.reshape(-1, 1))  # noiseless image obtained from phantom
# adding gaussian noise
# reconstructions using simple basis can deteriorate when noise is included
# escale = 10
# image = image + np.random.normal(0, escale, image.size).reshape(image.shape)

plt.figure()
plt.axis('off')
plt.imshow(image.reshape(resolution))
plt.colorbar(label='Signal [-]')
plt.show()

## Decomposition
Has to be done only once for a given basis and geometry matrix

Basis matrix should hold base vectors (simple, wavelets...) in columns

the choice of basis influences method's de-noising capability

In [ ]:
basis = sparse.eye(gmat.shape[1])  # simplest choice, no de-noising capability

Currently there are 3 implementations of BOB decomposition

 - Bob - standard implementation using numpy least squares method
 - SparseBob - using scipy.sparse that includes regularisation to avoid singular case in sparse matrix inversion
 - CholmodBob - sparse implementation using cholesky decomposition from sksparse useful for large matrices, includes simple regularisation to ensure positive definiteness

Regularisation can suppress oscillations caused by noise in insufficiently viewed areas of reconstruction plane.

In [ ]:
# bob = Bob()  # robust, faster for smaller and/or dense matrices
bob = SparseBob()  # faster for sparse matrices
# bob = CholmodBob()  # the fastest one, requires sksparse.cholmod

bob.decompose(gmat, basis, reg_factor=1e-8)

## Inversion

In [ ]:
res = bob(image)

In [ ]:
plt.figure()
plt.imshow(res.reshape(grid.shape), origin='lower', extent=grid.extent, vmin=-phantom.max(), vmax=phantom.max())
plt.colorbar(label='Emissivity [-]')
plt.xlabel('R [-]')
plt.ylabel('z [-]')
# plt.savefig('/figure/location/name.suffix')
plt.show()

In [ ]:
# retrofit
rf = gmat.dot(res)
plt.figure()
plt.imshow((rf-image).reshape(resolution))
plt.colorbar(label='Retrofit - Signal [-]')
plt.axis('off')
plt.show()

## Thresholded

In [ ]:
rest = bob.thresholding(image, c=3.1)

In [ ]:
plt.figure()
plt.imshow(rest.reshape(grid.shape), origin='lower', extent=grid.extent)
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.colorbar(label='Emissivity [-]')
plt.show()

In [ ]:
plt.figure()
plt.imshow(phantom - rest.reshape(grid.shape), origin='lower', extent=grid.extent)
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.colorbar(label='Phantom - Result [-]')
plt.show()

In [ ]:
rft = gmat.dot(rest)  # retrofit, forward model

In [ ]:
plt.figure()
plt.axis('off')
plt.imshow((rft-image).reshape(resolution))
plt.colorbar(label='Retrofit - Signal [-]')
plt.show()

### Total Recorded Intensity Check

In [ ]:
print('thresholded {}, full {}, image {} '.format(rft.sum(), rf.sum(), image.sum()))

## Saving decomposed matrices

In [ ]:
# bob.save_decomposition('path/to/decomposed_mat')